# Hacker Within survey analysis

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import custom data analysis module
from analyze_data import process_data, category_weights, get_categories

In [4]:
# Get data
df = pd.read_csv('thw_survey.csv')
df

key_tools-groups-yes  \
0                                      The Bash Shell   
1        Version Control / Git,Choosing a text editor   
2                               Version Control / Git   
3   The Bash Shell,Version Control / Git,Choosing ...   
4                              Choosing a text editor   
5                               Version Control / Git   
6                                      The Bash Shell   
7        Choosing a text editor,Version Control / Git   
8   Version Control / Git,Choosing a text editor,T...   
9                Version Control / Git,The Bash Shell   
10                              Version Control / Git   
11                                                NaN   
12                              Version Control / Git   
13              The Bash Shell,Choosing a text editor   
14                              Version Control / Git   
15  Version Control / Git,Choosing a text editor,T...   
16               The Bash Shell,Version Control / Git   
17                              Version Control / Git   
18               Version Control / Git,The Bash Shell   
19                             Choosing a text editor   
20               Version Control / Git,The Bash Shell   
21  Version Control / Git,The Bash Shell,Choosing ...   
22  Version Control / Git,The Bash Shell,Choosing ...   
23       Choosing a text editor,Version Control / Git   
24                              Version Control / Git   
25               Version Control / Git,The Bash Shell   
26                                                NaN   
27                                                NaN   
28                                     The Bash Shell   
29                                                NaN   

                   key_tools-groups-maybe  key_tools-ranks-yes-bash  \
0                  Choosing a text editor                       1.0   
1                          The Bash Shell                       NaN   
2                  Choosing a text editor                       NaN   
3                                     NaN                       1.0   
4    Version Control / Git,The Bash Shell                       NaN   
5                          The Bash Shell                       NaN   
6                                     NaN                       1.0   
7                                     NaN                       NaN   
8                                     NaN                       3.0   
9                  Choosing a text editor                       2.0   
10  Choosing a text editor,The Bash Shell                       NaN   
11                                    NaN                       NaN   
12                                    NaN                       NaN   
13                  Version Control / Git                       1.0   
14                         The Bash Shell                       NaN   
15                                    NaN                       3.0   
16                                    NaN                       1.0   
17  Choosing a text editor,The Bash Shell                       NaN   
18                 Choosing a text editor                       2.0   
19                  Version Control / Git                       NaN   
20                 Choosing a text editor                       2.0   
21                                    NaN                       2.0   
22                                    NaN                       2.0   
23                         The Bash Shell                       NaN   
24                                    NaN                       NaN   
25                 Choosing a text editor                       2.0   
26                                    NaN                       NaN   
27                                    NaN                       NaN   
28                  Version Control / Git                       1.0   
29                                    NaN                       NaN   

    key_tools-ranks-yes-version_control_git  \
0               

These data aren't very easy to summarize.  Y'all may remember that the questions were in five categories:

In [5]:
get_categories(df)

['category-key_tools',
 'category-languages',
 'category-practices',
 'category-libraries',
 'category-tools']

Each category has a set of options.  Here are the options for `category-key-tools`:

In [6]:
[col_name.split('-')[-1] for col_name in df.columns if col_name.startswith('key_tools-ranks-yes')]

['bash', 'version_control_git', 'choosing_text_editor']

For each category, you selected some options as "Yes" and some options as "Maybe".  The survey records these responds in two ways: as "groups" (a comma separated list of options) and as "ranks" (the rank for each option).  It records this twice, once for the "Yes" options, and once for the "Maybe" options.

Say there are `n` options in a category (n == 3 for category `key_tools`).  There will be 2 "group" columns for this category (group column for "Yes", group column for "Maybe".  There will be `n` "ranks" columns for "Yes" and `n` "ranks" columns for "Maybe".  That gives 2 + 2 * `n` columns in total, for each category.

In [7]:
key_tools_columns = [col_name for col_name in df.columns if col_name.startswith('key_tools')]
key_tools_columns                                                                                

['key_tools-groups-yes',
 'key_tools-groups-maybe',
 'key_tools-ranks-yes-bash',
 'key_tools-ranks-yes-version_control_git',
 'key_tools-ranks-yes-choosing_text_editor',
 'key_tools-ranks-maybe-bash',
 'key_tools-ranks-maybe-version_control_git',
 'key_tools-ranks-maybe-choosing_text_editor']

The ranks columns give the rank of this option for the "Yes" or "Maybe" sets.  Here are the column names for the ranks of the "Yes" set, `key_tools` category:

In [8]:
kt_y_rank_cols = [col_name for col_name in df.columns if col_name.startswith('key_tools-ranks-yes')]
kt_y_rank_cols

['key_tools-ranks-yes-bash',
 'key_tools-ranks-yes-version_control_git',
 'key_tools-ranks-yes-choosing_text_editor']

Here are "key tools" ranks, for the "Yes" set, for the first respondent:

In [9]:
df.loc[0, kt_y_rank_cols]

key_tools-ranks-yes-bash                      1
key_tools-ranks-yes-version_control_git     NaN
key_tools-ranks-yes-choosing_text_editor    NaN
Name: 0, dtype: object

The first respondent only put "The Bash" shell in the "Yes" set for this category.  Thence there are no ranks for the other two questions (the respondent didn't select them).

Last in the survey data we have five columns for the ranking you gave for each category, from 1 to 5.

In [10]:
df.columns[-5:]

Index(['category-key_tools', 'category-languages', 'category-practices',
       'category-libraries', 'category-tools'],
      dtype='object')

These data are a bit difficult to summarize, but hey, we're hackers, how hard can it be?

The analysis code is in the same directory as this Notebook, called `analyze_data.py`.

See that analysis code for details, but it generates:

* `cat_rank_mean` is the mean rank of each category (key tools, languages, etc);
* `cat_scores` is the composite score for each option, within category.  See below for what the composite scores are.
* `w_cat_scores` are the composite scores, weighted by the ranking of the matching category, given by each respondent.  A rank of 1 corresponds to a weight of 1, and a rank of 5 (out of 5 categories) corresponds to a weight of 1/5.

The composite scores are my attempt to weight the yes and the maybe ranks.  First we rescale the ranks as above, so a rank of 1 for an option corresponds to a rescaled score of 1, a rank of `n` (where `n` is the number of options in this category) gives a score of 1/`n`.   Next we add one, if this is a rank in the `yes` category, so a yes rank of 1 becomes 2 and 1/`n` becomes 1 + 1/`n`.  We then add the scores for the yes and the maybe ranks.  So, if there was only one repondent, giving a 1 rank for "bash shell" Yes, and 1 rank for "Choosing a text editor" Maybe, then the composite score is 1 + 1 == 2 for "bash shell", and 1 for "Choosing a text editor".

In [11]:
# Process data
cat_rank_mean, cat_scores, w_cat_scores = process_data(df)

In [12]:
# Some stuff for formatting output
pd.set_option('precision', 2)
from IPython.display import display, Markdown

def printmd(string):
    display(Markdown(string))


def disp_dict_row(d):
    display(pd.DataFrame(d, index=['']))

In [13]:
# Show the composite, weighted composite scores for each category.
for category in sorted(cat_scores):
    printmd('## Composite score for ' + category)
    disp_dict_row(cat_scores[category])
    printmd('## Weighted composite score for ' + category)
    disp_dict_row(w_cat_scores[category])
# Show the mean rank for each category.
printmd('## Category mean rank')
disp_dict_row(cat_rank_mean)

## Composite score for key_tools

bash  choosing_text_editor  version_control_git
  30.67                 26.67                42.67

## Weighted composite score for key_tools

bash  choosing_text_editor  version_control_git
  17.4                  13.8                23.93

## Composite score for languages

...

MEAN_software_stack  comparing_vc_software  django  docker  emacs  node.js  \
                10.12                  29.12   20.25   26.62   15.5    14.12   

  pandoc    vim  
   16.25  25.38

## Weighted composite score for tools

MEAN_software_stack  comparing_vc_software  django  docker  emacs  node.js  \
                 6.15                   15.4   12.12   16.33   8.93     8.68   

  pandoc    vim  
    9.65  15.97

## Category mean rank

key_tools  languages  libraries  practices  tools
       3.33       2.89       3.78       1.89   3.11

Rank all options in all categories on weighted score:

In [14]:
# Merge all the weighted option scores into a single dictionary.
all_weighted = {}
for key, sub_dict in w_cat_scores.items():
    prefix = key + '-'
    for sub_key, value in sub_dict.items():
        all_weighted[prefix + sub_key] = value
# Turn dictionary into sequence of (name, value) pairs, and sort descending by value.
all_vals = sorted(list(all_weighted.items()), key=lambda x : x[1], reverse=True)
# Display as a single column data frame.
names, values = zip(*all_vals)
display(pd.DataFrame(np.array(values), columns=['weighted score'], index=names))

weighted score
practices-visualization                    29.62
practices-continuous_integration           29.07
practices-testing                          28.24
languages-python                           26.38
languages-r                                24.78
key_tools-version_control_git              23.93
practices-reproducible                     23.33
practices-programming_paradigms            21.13
key_tools-bash                             17.40
practices-data_modes                       17.15
practices-gpu                              16.93
libraries-pandas                           16.50
tools-docker                               16.33
tools-vim                                  15.97
tools-comparing_vc_software                15.40
practices-bear_batch                       15.18
libraries-numpy                            14.40
languages-javascript                       13.85
key_tools-choosing_text_editor             13.80
practices-api_driven_development           13.67
libraries-data.table                       13.30
tools-django                               12.12
libraries-statistics                       10.80
practices-fast_databases                   10.40
practices-object_oriented                   9.91
tools-pandoc                                9.65
languages-julia                             9.25
tools-emacs                                 8.93
tools-node.js                               8.68
libraries-electron                          8.03
languages-matlab                            7.68
libraries-react.js                          7.53
languages-c                                 7.45
tools-MEAN_software_stack                   6.15
languages-fortran                           4.50
languages-java                              4.20